[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adasegroup/neural_prophet/blob/master/example_notebooks/hyperparameter_optimization_example.ipynb)

# Hyperparameter optimization with Ray Tune

We introduce the module for hyperparameter otpimization with Ray Tune.

It supports automatic tuning, with predefined by us hyperparameter sets, as well as manual tuning with user-provided configuration of the parameters.

Firstly, we will show how it works with NP model in automated mode.

In [4]:
if 'google.colab' in str(get_ipython()):
    data_location = "https://raw.githubusercontent.com/adasegroup/neural_prophet/master/"
    !pip install git+https://github.com/adasegroup/neural_prophet.git # may take a while
    !pip install tensorboardX
    !pip uninstall dataclasses -y
else:
    data_location = "../"

import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from neuralprophet.hyperparameter_tuner import tune_hyperparameters

df = pd.read_csv(data_location + "example_data/yosemite_temps.csv")
df.head(3)

,ds,y
0,2017-05-01 00:00:00,27.8
1,2017-05-01 00:05:00,27.0
2,2017-05-01 00:10:00,26.8


In [8]:
freq = '5min'
best_params, results_df = tune_hyperparameters('NP',
                                               df,
                                               freq)

In [5]:
best_params

{'growth': 'linear',
 'n_changepoints': 10,
 'changepoints_range': 0.5,
 'trend_reg': 10.0,
 'yearly_seasonality': False,
 'weekly_seasonality': True,
 'daily_seasonality': True,
 'seasonality_mode': 'additive',
 'seasonality_reg': 0.0,
 'n_lags': 30,
 'd_hidden': 8,
 'num_hidden_layers': 2,
 'ar_sparsity': 0.8,
 'learning_rate': 0.024150905458487568,
 'loss_func': 'Huber',
 'normalize': 'auto'}

This dictionary can further be used in initialization of NeuralProphet model.

This function has also additional parameters:
- **num_epochs**: Max possible number of epochs to train each model.
- **num_samples**: Number of samples from hyperparameter spaces to check.
- **resources_per_trial**: Resources per trial setting for ray.tune.run, {'cpu': 1, 'gpu': 2} for example



This function additionally will output the dataframe with detailed result of each trial if return_results is set to True.

In [7]:
results_df[['config.growth', 'config.n_changepoints', 'config.changepoints_range',
       'config.trend_reg', 'config.yearly_seasonality',
       'config.weekly_seasonality', 'config.daily_seasonality',
       'config.seasonality_mode', 'config.seasonality_reg', 'config.n_lags',
       'config.d_hidden', 'config.num_hidden_layers', 'config.ar_sparsity',
       'config.learning_rate', 'config.loss_func', 'config.normalize']].head()

,config.growth,config.n_changepoints,config.changepoints_range,config.trend_reg,config.yearly_seasonality,config.weekly_seasonality,config.daily_seasonality,config.seasonality_mode,config.seasonality_reg,config.n_lags,config.d_hidden,config.num_hidden_layers,config.ar_sparsity,config.learning_rate,config.loss_func,config.normalize
trial_id,,,,,,,,,,,,,,,,
8f12d_00000,off,10,0.9,10.0,False,False,True,multiplicative,1.0,100,64,8,0.8,0.000733,MSE,off
8f12d_00001,linear,100,0.8,0.5,True,False,True,additive,0.0,10,128,16,0.8,0.019590,MSE,soft
8f12d_00002,linear,10,0.5,0.0,True,False,False,multiplicative,0.0,30,128,2,0.1,0.000306,MSE,standardize
8f12d_00003,linear,10,0.5,10.0,True,True,True,multiplicative,0.5,30,8,8,0.8,0.000195,Huber,off
8f12d_00004,linear,10,0.5,0.5,True,True,True,additive,0.5,10,64,2,0.8,0.008229,MSE,soft


## Manual mode

In case of manual mode, a user must provide a config dictionary with hyperparameter spaces in compatability with Ray Tune api.

We provide a minimal example below, for more information on Search Spaces withit this link https://docs.ray.io/en/master/tune/api_docs/search_space.html?highlight=tune.choice

In [5]:
from ray import tune

config = {'n_lags': tune.grid_search([10, 20, 30]),
          'learning_rate': tune.loguniform(1e-4, 1e-1),
          'num_hidden_layers': tune.choice([2, 8, 16])}

In [10]:
freq = '5min'
best_params, results_df = tune_hyperparameters('NP', 
                                               df,
                                               freq, 
                                               mode = 'manual',
                                               num_epochs = 10,
                                               num_samples = 3,
                                              config = config)

In [8]:
best_params

{'n_lags': 10, 'learning_rate': 0.07003935175029312, 'num_hidden_layers': 2}

In [7]:
results_df

,loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.n_lags,config.learning_rate,config.num_hidden_layers
trial_id,,,,,,,,,,,,,,,,,,,,
3a812_00000,0.000070,0.834731,True,None,None,100,7016b6970e7646989f1ad31fa10adc43,2021-05-22_01-40-07,1621636807,108.165445,45197,MacBook-Pro-Polina.local,192.168.0.7,108.165445,0,100,"0_learning_rate=0.070039,n_lags=10,num_hidden_...",10,0.070039,2
3a812_00001,0.476743,1.610536,True,None,None,10,c7ba48b1b54b4f1389f74c391e2d5a49,2021-05-22_01-38-40,1621636720,21.097708,45200,MacBook-Pro-Polina.local,192.168.0.7,21.097708,0,10,"1_learning_rate=0.0080468,n_lags=20,num_hidden...",20,0.008047,2
3a812_00002,0.728775,1.772149,True,None,None,10,6df8fd44bae84eea94d3f52f9d0c6bfe,2021-05-22_01-38-49,1621636729,29.713108,45201,MacBook-Pro-Polina.local,192.168.0.7,29.713108,0,10,"2_learning_rate=0.00010565,n_lags=30,num_hidde...",30,0.000106,16
3a812_00003,0.458268,1.635460,True,None,None,10,0b8ecaef670149ee868509233a9d0fba,2021-05-22_01-38-40,1621636720,20.723858,45198,MacBook-Pro-Polina.local,192.168.0.7,20.723858,0,10,"3_learning_rate=0.0090063,n_lags=10,num_hidden...",10,0.009006,2
3a812_00004,0.000216,1.092426,True,None,None,20,ccab8955200f49a999603047ed743429,2021-05-22_01-38-57,1621636737,37.349945,45196,MacBook-Pro-Polina.local,192.168.0.7,37.349945,0,20,"4_learning_rate=0.062134,n_lags=20,num_hidden_...",20,0.062134,8
3a812_00005,0.324455,1.523596,True,None,None,10,7362e63a078e40549f5b1896b6912edb,2021-05-22_01-38-40,1621636720,20.600341,45202,MacBook-Pro-Polina.local,192.168.0.7,20.600341,0,10,"5_learning_rate=0.013602,n_lags=30,num_hidden_...",30,0.013602,2
3a812_00006,0.982963,1.588013,True,None,None,10,057284675341458790f22671ef20824e,2021-05-22_01-38-40,1621636720,21.107830,45199,MacBook-Pro-Polina.local,192.168.0.7,21.107830,0,10,"6_learning_rate=0.0020884,n_lags=10,num_hidden...",10,0.002088,2
3a812_00007,0.739106,1.784202,True,None,None,10,72cb54b89dc644b7a6bc2afa5a18e4e6,2021-05-22_01-38-48,1621636728,28.905643,45195,MacBook-Pro-Polina.local,192.168.0.7,28.905643,0,10,"7_learning_rate=0.00092774,n_lags=20,num_hidde...",20,0.000928,16
3a812_00008,0.814214,1.976280,True,None,None,10,e743dda719bf4417ad9312453d6123cb,2021-05-22_01-39-00,1621636740,16.456889,45235,MacBook-Pro-Polina.local,192.168.0.7,16.456889,0,10,"8_learning_rate=0.0023869,n_lags=30,num_hidden...",30,0.002387,16


# Other models

Hyperparameter optimization can be also run for each of 4 models, available in Neuralprophet library: LSTM, NBeats, TFT and DeepAR. In order to run optimization for particular model, the procedure is the same, you just need to insert the name of preferable model.

An example for LSTM can be found below:

In [9]:
freq = '5min'
best_params, results_df = tune_hyperparameters('LSTM',
                                               df,
                                               freq)

In [9]:
best_params

{'learning_rate': 0.013718466327407178,
 'd_hidden': 128,
 'n_lags': 100,
 'num_hidden_layers': 2,
 'lstm_bias': True,
 'lstm_bidirectional': False}

In [10]:
results_df.head()

,loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,...,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.learning_rate,config.d_hidden,config.n_lags,config.num_hidden_layers,config.lstm_bias,config.lstm_bidirectional
trial_id,,,,,,,,,,,,,,,,,,,,,
ddb98_00000,0.047840,184.533827,False,NaN,NaN,8.0,a26094effe6246d5af6b147ae60a5933,2021-05-21_14-47-36,1.621598e+09,1486.938134,...,1486.938134,0.0,8.0,"0_d_hidden=128,learning_rate=0.000337,lstm_bia...",0.000337,128.0,30.0,16.0,False,True
ddb98_00001,0.051339,48.592918,True,NaN,NaN,20.0,4ebbb108e09e4b2986c3835634ed0295,2021-05-21_14-38-22,1.621597e+09,932.583967,...,932.583967,0.0,20.0,"1_d_hidden=8,learning_rate=0.0097144,lstm_bias...",0.009714,8.0,100.0,8.0,False,True
ddb98_00002,0.255517,28.091819,True,NaN,NaN,10.0,4be3033d37004d4f98c47a2d0d564c11,2021-05-21_14-27-36,1.621596e+09,285.940451,...,285.940451,0.0,10.0,"2_d_hidden=64,learning_rate=0.00010157,lstm_bi...",0.000102,64.0,100.0,8.0,False,False
ddb98_00003,0.000029,11.933487,True,NaN,NaN,100.0,44266641f7124ffda7d62dbdab2b0b49,2021-05-21_14-43-17,1.621597e+09,1227.605633,...,1227.605633,0.0,100.0,"3_d_hidden=128,learning_rate=0.013718,lstm_bia...",0.013718,128.0,100.0,2.0,True,False
ddb98_00004,0.047503,91.865763,False,NaN,NaN,16.0,f6f09c2bc638437498bd2d01fd44961d,2021-05-21_14-47-51,1.621598e+09,1501.235601,...,1501.235601,0.0,16.0,"4_d_hidden=8,learning_rate=0.0013775,lstm_bias...",0.001378,8.0,100.0,16.0,False,True
